In [ ]:
import configparser
import pickle
import time
import cv2
import os
import numpy as np
from datetime import datetime
from pathlib import Path

import imutils
import face_recognition
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [ ]:
# find path of xml file containing haarcascade file
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc_live', "rb").read())

In [ ]:
# trained faces
names = sorted(list(set(data['names'])))
for name in names:
    print(name, '\n')

In [ ]:
def init_spreadsheet_service():
  SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

  """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.pickle stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  token_file = Path(FILEDIR, 'token.pickle')
  creds_file = Path(FILEDIR, 'credentials.json')
  if os.path.exists(token_file):
    with open(token_file, 'rb') as token:
      creds = pickle.load(token)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          creds_file, SCOPES)
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(token_file, 'wb') as token:
      pickle.dump(creds, token)

  service = build('sheets', 'v4', credentials=creds)
  return service


def create_new_spreadsheet(spreadsheet_name):
  # Create spreadsheets service
  service = init_spreadsheet_service()
  spreadsheet = {
      'properties': {
          'title': spreadsheet_name
      }
  }
  spreadsheet = service.spreadsheets().create(body=spreadsheet,
                                              fields='spreadsheetId').execute()
  return spreadsheet.get('spreadsheetId')


def get_data(spreadsheet_id, range_name='Sheet1'):
  service = init_spreadsheet_service()
  result = service.spreadsheets().values().get(
      spreadsheetId=spreadsheet_id, range=range_name).execute()
  rows = result.get('values', [])
  return rows


def initiate_header(spreadsheet_id, range_name='Sheet1'):
  prev_data = get_data(spreadsheet_id)
  if len(prev_data) > 0:
    return
  elif len(prev_data) == 0:
    data = [['Person', 'Recorded Time']]
    service = init_spreadsheet_service()

    body = {
        'values': [i for i in data]
    }

    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='RAW', body=body).execute()
    return


def check_for_new_run(config_file, spreadsheet_name):
  config = configparser.ConfigParser()
  config.read(config_file)

  current_date = datetime.today().strftime("%d-%m-%Y")
  config_date = config['base']['current_date']
  if config_date.strip() == current_date.strip():
    print(
        "Script already run today, not creating a new spreadsheet")
    spreadsheet_id = config['base']['spreadsheet_id']
    print(f"Spreadsheet ID: {spreadsheet_id}")
  else:
    print(
        f"Script last run on {config_date}. Date today: {current_date}")
    print("Creating new spreadsheet")
    spreadsheet_id = create_new_spreadsheet(spreadsheet_name)
    config['base']['spreadsheet_id'] = spreadsheet_id
    config['base']['current_date'] = current_date
    print(f"Spreadsheet ID: {spreadsheet_id}")
  
  # check if header exists and create otherwise
  initiate_header(spreadsheet_id)
  with open(config_file, 'w') as conf:
    config.write(conf)
  return spreadsheet_id


def upload_data(spreadsheet_id, data, range_name='Sheet1'):
  service = init_spreadsheet_service()

  body = {
      'values': [i for i in data]
  }

  service.spreadsheets().values().append(
      spreadsheetId=spreadsheet_id, range=range_name,
      valueInputOption='RAW', body=body).execute()


def upload_faces(spreadsheet_id, faces):
#   prev_entries = get_spreadsheet_data(spreadsheet_id)
  current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#   prev_data = get_data(spreadsheet_id)
  all_data = [[i, current_time] for i in faces]
  print("Uploading faces -", ", ".join(faces), "- at time", current_time)
  upload_data(spreadsheet_id, all_data)

In [ ]:
FILEDIR = os.path.abspath('')
CONFIG_FILE = Path(FILEDIR, 'smart_security_config.ini')

In [ ]:
# Initiate spreadsheet
spreadsheet_name = "Smart Security %s" % (datetime.today().strftime("%d-%m-%Y"))
spreadsheet_id = check_for_new_run(CONFIG_FILE, spreadsheet_name)

In [ ]:
print("Streaming started")
try:
  video_capture = cv2.VideoCapture(0)
  # loop over frames from the video file stream
  while True:
    # grab the frame from the threaded video stream
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    # convert the input frame from BGR to RGB 
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    for encoding in encodings:
     #Compare encodings with encodings in data["encodings"]
     #Matches contain array with boolean values and True for the embeddings it matches closely
     #and False for rest
      matches = face_recognition.compare_faces(data["encodings"],
       encoding)
      #set name =inknown if no encoding matches
      name = "Unknown"
      # check to see if we have found a match
      if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
          #Check the names at respective indexes we stored in matchedIdxs
          name = data["names"][i]
          #increase count for the name we got
          counts[name] = counts.get(name, 0) + 1
        #set name which has highest count
        name = max(counts, key=counts.get)

      # update the list of names
      names.append(name)
      upload_faces(spreadsheet_id, names)
finally:
  video_capture.release()
  cv2.destroyAllWindows()